Chanisorn/thai-food-mpnet-new-v8

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import InformationRetrievalEvaluator
import pandas as pd

model = SentenceTransformer("Chanisorn/thai-food-mpnet-new-v8")

test_df_raw = pd.read_csv("test set 2 - test_set_for_google_sheets.csv")  # update path if needed

# Expand test set with anchor-positive pairs
test_rows = []

for _, row in test_df_raw.iterrows():
    dish = row["dish"]
    for col in ["query1 อยากกินอาหารครบ", "query2", "query3"]:
        query = row[col]
        if pd.notna(query):
            test_rows.append({"anchor": query.strip(), "positive": dish.strip()})

test_df = pd.DataFrame(test_rows)

# Prepare data for evaluator
queries = {f"q{i}": row["anchor"] for i, row in test_df.iterrows()}
corpus = {f"d{i}": row["positive"] for i, row in test_df.iterrows()}
relevant_docs = {f"q{i}": {f"d{i}"} for i in range(len(test_df))}

# Evaluate
evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    name="thai-food-test-eval",
    show_progress_bar=True,
    precision_recall_at_k=[1, 3, 5]
)

evaluator(model)


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:04<00:00,  4.79s/it]


{'thai-food-test-eval_cosine_accuracy@1': 0.2585034013605442,
 'thai-food-test-eval_cosine_accuracy@3': 0.7074829931972789,
 'thai-food-test-eval_cosine_accuracy@5': 0.7891156462585034,
 'thai-food-test-eval_cosine_accuracy@10': 0.8435374149659864,
 'thai-food-test-eval_cosine_precision@1': 0.2585034013605442,
 'thai-food-test-eval_cosine_precision@3': 0.23582766439909295,
 'thai-food-test-eval_cosine_precision@5': 0.15782312925170064,
 'thai-food-test-eval_cosine_recall@1': 0.2585034013605442,
 'thai-food-test-eval_cosine_recall@3': 0.7074829931972789,
 'thai-food-test-eval_cosine_recall@5': 0.7891156462585034,
 'thai-food-test-eval_cosine_ndcg@10': 0.5661307373640255,
 'thai-food-test-eval_cosine_mrr@10': 0.4745653817082389,
 'thai-food-test-eval_cosine_map@100': 0.4812900188022347}

ดู result ของ Chanisorn/thai-food-mpnet-new-v8 กับ test set

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

model = SentenceTransformer("Chanisorn/thai-food-mpnet-new-v8")

df = pd.read_csv("test set 2 - test_set_for_google_sheets.csv") 
corpus_dishes = df["dish"].tolist()

# Embedding 
dish_embeddings = model.encode(corpus_dishes, convert_to_tensor=True)

# Loop query
for i, row in df.iterrows():
    for col in ["query1 อยากกินอาหารครบ", "query2", "query3"]:  
        query = row[col]
        if pd.notna(query):
            # Embedding for  query
            query_embedding = model.encode(query, convert_to_tensor=True)
            cos_scores = util.cos_sim(query_embedding, dish_embeddings)[0]
            top_k = min(3, len(corpus_dishes))
            top_results = torch.topk(cos_scores, k=top_k)

            print(f"\n💬 Query: {query}")
            print("🍽️ Top matched dishes:")
            for idx in top_results.indices:
                dish = corpus_dishes[idx.item()]
                score = cos_scores[idx].item()
                print(f"- {dish} (score: {score:.4f})")



💬 Query: มีปลานิลเหลือๆ มีพริกแกง ใบมะกรูด ทำอะไรดี
🍽️ Top matched dishes:
- ฉู่ฉี่ปลานิล (สูตรคุณยาย) (score: 0.6092)
- น้ำจิ้มปลาเผา (มะขาม) (score: 0.4494)
- ขนมจีนแกงไตปลา (score: 0.4416)

💬 Query: อยากกินอาหารบ้าน ๆ สูตรคุณยาย
🍽️ Top matched dishes:
- น้ำพริกกะปิสูตรคุณยาย (score: 0.4577)
- ฉู่ฉี่ปลานิล (สูตรคุณยาย) (score: 0.3835)
- ลาบมาม่า (score: 0.2790)

💬 Query: มีพริกแกงอยู่ มีปลาด้วย ทำอะไรดี
🍽️ Top matched dishes:
- ขนมจีนแกงไตปลา (score: 0.5272)
- น้ำพริกกะปิสูตรคุณยาย (score: 0.5226)
- ผัดพริกหยวกหมู (score: 0.4791)

💬 Query: อยากอีสเมี่ยงคำ
🍽️ Top matched dishes:
- เส้นหมี่ผัดซีอิ๊วหมูนุ่ม (score: 0.4144)
- ไส้กรอกซอสมะขาม (score: 0.4006)
- วุ้นเส้นผัดไข่ (score: 0.3978)

💬 Query: อยากทำเมี่ยงคำกินเล่น ใช้ใบชะพลู
🍽️ Top matched dishes:
- บานอฟฟี (score: 0.4537)
- สลัดแร็ปผัก (score: 0.3928)
- ก๋วยเตี๋ยวหลอดทรงเครื่อง (score: 0.3683)

💬 Query: มีขิง หอมแดง มะนาว ถั่ว กุ้งแห้ง อยากห่อเมี่ยงคำ
🍽️ Top matched dishes:
- ข้าวมันกุ้ง (score: 0.4529)
- วุ้นกะทิมะพร้าวน้ำหอม (

In [ ]:
paraphrase-multilingual-MiniLM-L12-v2

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import InformationRetrievalEvaluator
import pandas as pd

# 🔧 Load your pretrained model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 📄 Load your test set CSV (with query1–3 and ชื่ออาหาร)
test_df_raw = pd.read_csv("test set 2 - test_set_for_google_sheets.csv")  # update path if needed

# 📌 Expand test set with anchor-positive pairs
test_rows = []

for _, row in test_df_raw.iterrows():
    dish = row["dish"]
    for col in ["query1 อยากกินอาหารครบ", "query2", "query3"]:
        query = row[col]
        if pd.notna(query):
            test_rows.append({"anchor": query.strip(), "positive": dish.strip()})

test_df = pd.DataFrame(test_rows)

# ✅ Prepare data for evaluator
queries = {f"q{i}": row["anchor"] for i, row in test_df.iterrows()}
corpus = {f"d{i}": row["positive"] for i, row in test_df.iterrows()}
relevant_docs = {f"q{i}": {f"d{i}"} for i in range(len(test_df))}

# 🧪 Evaluate
evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    name="thai-food-test-eval",
    show_progress_bar=True,
    precision_recall_at_k=[1, 3, 5]
)

evaluator(model)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:01<00:00,  1.41s/it]


{'thai-food-test-eval_cosine_accuracy@1': 0.08163265306122448,
 'thai-food-test-eval_cosine_accuracy@3': 0.2653061224489796,
 'thai-food-test-eval_cosine_accuracy@5': 0.3741496598639456,
 'thai-food-test-eval_cosine_accuracy@10': 0.5034013605442177,
 'thai-food-test-eval_cosine_precision@1': 0.08163265306122448,
 'thai-food-test-eval_cosine_precision@3': 0.08843537414965986,
 'thai-food-test-eval_cosine_precision@5': 0.07482993197278912,
 'thai-food-test-eval_cosine_recall@1': 0.08163265306122448,
 'thai-food-test-eval_cosine_recall@3': 0.2653061224489796,
 'thai-food-test-eval_cosine_recall@5': 0.3741496598639456,
 'thai-food-test-eval_cosine_ndcg@10': 0.27316877919492205,
 'thai-food-test-eval_cosine_mrr@10': 0.20124446604038443,
 'thai-food-test-eval_cosine_map@100': 0.21374266219769097}

ดู result ของ paraphrase-multilingual-MiniLM-L12-v2 กับ test set

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

df = pd.read_csv("test set 2 - test_set_for_google_sheets.csv")  
corpus_dishes = df["dish"].tolist()

# Embedding
dish_embeddings = model.encode(corpus_dishes, convert_to_tensor=True)

# Loop query
for i, row in df.iterrows():
    for col in ["query1 อยากกินอาหารครบ", "query2", "query3"]:  
        query = row[col]
        if pd.notna(query):
            # Embedding for query
            query_embedding = model.encode(query, convert_to_tensor=True)
            cos_scores = util.cos_sim(query_embedding, dish_embeddings)[0]
            top_k = min(3, len(corpus_dishes))
            top_results = torch.topk(cos_scores, k=top_k)

            print(f"\n💬 Query: {query}")
            print("🍽️ Top matched dishes:")
            for idx in top_results.indices:
                dish = corpus_dishes[idx.item()]
                score = cos_scores[idx].item()
                print(f"- {dish} (score: {score:.4f})")


💬 Query: มีปลานิลเหลือๆ มีพริกแกง ใบมะกรูด ทำอะไรดี
🍽️ Top matched dishes:
- ผัดกะเพรากุ้ง (score: 0.7203)
- ต้มยำปลาคัง (score: 0.7038)
- ขนมจีนแกงไตปลา (score: 0.6955)

💬 Query: อยากกินอาหารบ้าน ๆ สูตรคุณยาย
🍽️ Top matched dishes:
- น้ำพริกกะปิสูตรคุณยาย (score: 0.6965)
- ฉู่ฉี่ปลานิล (สูตรคุณยาย) (score: 0.5733)
- น้ำพริกกากหมู (score: 0.5563)

💬 Query: มีพริกแกงอยู่ มีปลาด้วย ทำอะไรดี
🍽️ Top matched dishes:
- ต้มยำปลาคัง (score: 0.7900)
- ผัดกะเพรากุ้ง (score: 0.7621)
- ขนมจีนแกงไตปลา (score: 0.7417)

💬 Query: อยากอีสเมี่ยงคำ
🍽️ Top matched dishes:
- เมี่ยงคำ (score: 0.8729)
- ต้มเล้งแซ่บ (score: 0.7523)
- ลาบมาม่า (score: 0.7472)

💬 Query: อยากทำเมี่ยงคำกินเล่น ใช้ใบชะพลู
🍽️ Top matched dishes:
- ก๋วยเตี๋ยวหลอดทรงเครื่อง (score: 0.7350)
- แกงป่าไก่ (score: 0.7082)
- ก๋วยเตี๋ยวเนื้อสับ (score: 0.6939)

💬 Query: มีขิง หอมแดง มะนาว ถั่ว กุ้งแห้ง อยากห่อเมี่ยงคำ
🍽️ Top matched dishes:
- ผัดกะเพรากุ้ง (score: 0.6716)
- ไข่เจียวทูน่าพริกสับ (score: 0.6561)
- กุ้งกระเบื้องสาหร่าย (score